In [ ]:
from robotic import ry

In [ ]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view()

In [ ]:
stick = C.addFrame('box','table') \
    .setShape(ry.ST.capsule, size=[1.,.03]) \
    .setColor([1,.5,0])
C.view()

In [ ]:
#change pose here and re-optimize
stick.setRelativePose('t(-.4 .2 1.3) d(-40 0 1 0)')
C.view()

In [ ]:
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'box'], ry.OT.eq, [[1e1, 0, 0],[0, 1e1, 0]]);
komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'box'], ry.OT.ineq, [[0, 0, -1e1]], [0,0,-.5]);
komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'box'], ry.OT.ineq, [[0, 0, 1e1]], [0,0,.5]);
komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', 'box'], ry.OT.eq, [1e1], [0]);
komo.addObjective([], ry.FS.distance, ['l_palm', 'box'], ry.OT.ineq, [1e1]);

In [ ]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

In [ ]:
#komo.view(False, "IK solution")
q = komo.getPath()
C.setJointState(q[0])
C.view(False, "IK solution")

In [ ]:
del komo
del C